In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
new_directory = '/content/drive/MyDrive/MadridWeek4'
os.chdir(new_directory)

updated_directory = os.getcwd()
print("更新后的工作目录:", updated_directory)


更新后的工作目录: /content/drive/MyDrive/MadridWeek4


In [3]:
import json
import os
import moviepy
from moviepy.editor import *
from moviepy import *
import shutil
import numpy as np
from datetime import datetime

In [4]:
# this function loads all films in the folder given as argument into the 'input' file (make sure you are in the right
# directory)

def loadVideosToInfile(folder):
    videos = os.listdir(folder)
    with open('input', 'w') as inputfile:
        for v in videos:
            if v.split('.')[-1] == 'mp4':
                inputfile.write(v+'\n')

In [5]:
# This function should read all .json files in the output folder, and combine them into one large dictionary
def readAllOutput(outputFolder):
    dictionaries = {}
    output_files = os.listdir(outputFolder)
    for f in output_files:
        if f.split('.')[-1]=='json':
            file = open(os.path.join(outputFolder,f))
            segments = json.load(file)
            for s in segments:
                dictionaries[s['video']] = s

    return dictionaries

In [90]:
def searchForMatch(keyFilmName, featureDictionary, frame=None, fps=25, nBestMatches=1, average=False, mode='features'):
    bestRangeStart = 0
    bestRangeFilmName = ''
    bestRangeScore = 1000000000

    fragments = []

    keyClips = featureDictionary[keyFilmName]['clips']
    keyFeatures = []
    keyLabels = []

    for c in keyClips:
        if 'label' in c:
            keyLabels.append(c['label'])
            if mode in c:
                keyFeatures.append(np.array(c[mode]))
            else:
                # 处理找不到特定模式特征的情况
                print(f"Warning: No features found for mode '{mode}' in key clip of '{keyFilmName}'.")
        else:
            # 处理找不到标签的情况
            print(f"Warning: No label found in key clip of '{keyFilmName}'.")

    # 确保 keyFeatures 不为空
    if not keyFeatures:
        print("Error: keyFeatures is empty.")
        return fragments

    # Add print statements for debugging
    print("keyClips:", keyClips)
    print("keyLabels:", keyLabels)
    print("keyFeatures:", keyFeatures)
    print("len(keyFeatures):", len(keyFeatures))

    # 计算距离和平均值
    for film in featureDictionary.keys():
        if not film == keyFilmName:
            filmClips = featureDictionary[film]['clips']
            filmFeatures = []
            for c in filmClips:
                filmFeatures.append(np.array(c[mode]))

            for i in range(len(filmFeatures) - len(keyFeatures)):
                distance = 0  # 1000000000 #0
                for j in range(len(keyFeatures)):
                    d = np.linalg.norm(filmFeatures[i + j] - keyFeatures[j])
                    # if d < distance:
                    #     distance = d
                    distance += d

                distance = distance / len(keyFeatures)

                if average:
                    avgFeatureRange = filmFeatures[i:i + len(keyFeatures)]
                    avgRef = np.average(avgFeatureRange, axis=0)
                    distance = np.linalg.norm(averageFeature - avgRef)

                a = fragmentExists(fragments, [film, i, len(keyFeatures), distance])
                if a:
                    fragments = a
                elif len(fragments) < nBestMatches:
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key=lambda x: x[-1])
                elif distance < fragments[-1][-1]:
                    fragments.pop()
                    fragments.append([film, i, len(keyFeatures), distance])
                    fragments.sort(key=lambda x: x[-1])

    # 将 return fragments 移到函数内部
    return fragments


In [8]:
def fragmentExists(fragments, nf):
    for f in fragments:
        r = range(nf[1], nf[1]+nf[2])
        if f[0] == nf[0] and (f[1] in r or f[1]+f[2] in r):
            if f[-1] > nf[-1]:
                fragments.remove(f)
                fragments.append(nf)
                fragments.sort(key = lambda x : x[-1])
            return fragments
    return False

In [102]:
# this is a helper function to which you give the fragment (the output of the searchForMatch function), and it returns
# a moviepy film object (which you can then save)

def extractFragment(fragment, filmFolder):
    filmFileName = fragment[0]
    start = fragment[1]
    nrClips = fragment[2]

    film = VideoFileClip(os.path.join(filmFolder, filmFileName))
    tsfps = film.fps
    tsFrameLength = 1/tsfps

    nrFrames = nrClips*32
    startFrame = start*32

    startTime = startFrame*tsFrameLength
    endTime = startTime+nrFrames*tsFrameLength

    startFrame = min(startFrame, film.reader.nframes - nrFrames)
    startTime = startFrame * tsFrameLength
    endTime = startTime + nrFrames * tsFrameLength
    """if endTime < film.duration:
        endTime = film.duration"""

    #path = getPoC(filmFileName, '/Volumes/28_PoC/meteora\'s_friends')
    #film = VideoFileClip(path)
    filmFragment = film.subclip(startTime,endTime)

    return filmFragment

    # This function should read all .json files in the output folder, and combine them into one large dictionary

In [13]:
def getPoC(file, directory):
    name = file.split('.')[0]
    film = os.path.join(directory,name)
    filmFiles = os.listdir(film)
    path = ''
    for f in filmFiles:
        if f.split('.')[0] == 'movie':
            path = os.path.join(film,f)
            return path

In [14]:
# call function to load the filenames to 'input'
loadVideosToInfile('input_films')

In [65]:
# run the model to process all films currently in the input_films folder, and save it to a json file, here named by the date

now = datetime.now()
dt_string = now.strftime("%Y_%m_%d_%H_%M")
print('**********************')
print('**********************')
print('start: ', dt_string)
print('**********************')
print('**********************')


MODE = 'score'
filename = 'output/output'+dt_string+'_'+MODE+'.json'

!python /content/drive/MyDrive/MadridWeek3/video-classification-3d-cnn-pytorch/main.py --input input --video_root input_films --output {filename} --model ../SkillClass1/video-classification-3d-cnn-pytorch/resnet-34-kinetics-cpu.pth --mode {MODE} --no_cuda

then = datetime.now()
dt_string = then.strftime("%Y_%m_%d_%H_%M")
print('**********************')
print('**********************')
print('end: ', dt_string)
print('duration: ', then-now)
print('**********************')
print('**********************')

**********************
**********************
start:  2023_11_25_17_31
**********************
**********************
^C
**********************
**********************
end:  2023_11_25_17_31
duration:  0:00:02.849272
**********************
**********************


In [17]:
filename

'output/output2023_11_25_16_41_score.json'

In [18]:
# a small piece of code that moves all processed input films to the processed films folder

input_films_processed = os.listdir('input_films')
print(input_films_processed)
for file in input_films_processed:
    try:
        shutil.move(os.path.join('input_films',file),'processed_films')
    except:
        print(file + ' not moved')

[]


In [19]:
# read all output with the function declared above

dicts = readAllOutput('output')

In [20]:
dicts.keys()

dict_keys(['frame-112191.mp4', 'frame-005099.mp4', 'frame-004388.mp4', 'frame-108488.mp4', 'frame-092035.mp4', 'frame-064728.mp4', 'frame-079506.mp4', 'frame-079279.mp4', 'frame-068430.mp4', 'frame-011017.mp4', 'frame-050353.mp4', 'frame-061214.mp4', 'frame-055097.mp4'])

In [21]:
keyClips = dicts['frame-112191.mp4']['clips']

In [22]:
keyClips

[{'segment': [1, 16],
  'label': 'tap dancing',
  'scores': [0.39507219195365906,
   2.1236419677734375,
   0.7308951616287231,
   1.677464485168457,
   -1.178391456604004,
   0.9293416738510132,
   0.266100138425827,
   -0.19055336713790894,
   -0.5772504210472107,
   0.3296527862548828,
   -2.557180643081665,
   -0.3566390573978424,
   -1.4750056266784668,
   -2.00931978225708,
   1.1900830268859863,
   2.1330111026763916,
   3.6222331523895264,
   0.3225516080856323,
   3.0571746826171875,
   -1.5126712322235107,
   0.7227524518966675,
   -0.30291417241096497,
   -0.7970036268234253,
   0.5525384545326233,
   1.335798740386963,
   -1.2710529565811157,
   -0.2188563346862793,
   -0.21091926097869873,
   1.275160312652588,
   -0.7258147597312927,
   0.9187740087509155,
   1.579329252243042,
   -1.8714854717254639,
   0.06533358246088028,
   3.2113919258117676,
   -1.0119222402572632,
   -1.3378310203552246,
   -1.1801493167877197,
   -1.109752893447876,
   0.5883309841156006,
   0.115

In [82]:
keyClips = dicts['frame-112191.mp4']['clips']
keyFeatures =[]
keyLabels =[]
mode = 'scores'

for c in keyClips:
  keyFeatures.append(np.array(c[mode]))
  keyLabels.append(c['label'])
  print(keyFeatures)



Streaming output truncated to the last 5000 lines.
       -1.01192224, -1.33783102, -1.18014932, -1.10975289,  0.58833098,
        0.11524142,  2.29657149, -2.17276049,  1.61292958, -1.04042637,
        1.16654706, -0.55155355, -1.95077062, -0.57443821, -0.54026872,
       -1.25074434,  1.03128445, -1.4780581 , -0.87345904, -2.40288472,
       -0.48704869,  0.92638373,  1.35473669, -1.56689036, -1.75597477,
        0.13865632, -0.35425293, -0.99508262,  0.04017928,  0.30224186,
        1.06223142, -0.92999828,  1.23254681, -0.10785562,  2.25691199,
       -0.76278454, -0.02574287, -1.39543414, -1.50874567,  0.41867518,
        0.97209442, -2.0386157 , -0.2674669 , -2.8832798 ,  0.56552196,
       -1.78920078, -2.51394057, -0.35150605, -0.53568989,  3.60708857,
        5.1766839 , -0.03024411,  0.32327539, -1.2166816 , -1.23384917,
        0.35841489,  2.29865313,  0.2642099 , -0.09505361, -0.84390259,
       -1.12392545, -0.41992754, -2.16287923,  0.37643778, -0.05994196,
       -0.158

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [89]:
len(keyFeatures)

62

In [95]:
# look for the best matching sequence to the input file (needs to be in dicts for it to work)

keyName = 'frame-112191.mp4'
film = VideoFileClip(os.path.join('processed_films', keyName))
frPS = film.fps

s = 0
d = 8

fragment = searchForMatch(keyName, dicts, frame = [s,s+d], fps = frPS, nBestMatches = 10, average = False, mode = 'scores')

keyClips: [{'segment': [1, 16], 'label': 'tap dancing', 'scores': [0.39507219195365906, 2.1236419677734375, 0.7308951616287231, 1.677464485168457, -1.178391456604004, 0.9293416738510132, 0.266100138425827, -0.19055336713790894, -0.5772504210472107, 0.3296527862548828, -2.557180643081665, -0.3566390573978424, -1.4750056266784668, -2.00931978225708, 1.1900830268859863, 2.1330111026763916, 3.6222331523895264, 0.3225516080856323, 3.0571746826171875, -1.5126712322235107, 0.7227524518966675, -0.30291417241096497, -0.7970036268234253, 0.5525384545326233, 1.335798740386963, -1.2710529565811157, -0.2188563346862793, -0.21091926097869873, 1.275160312652588, -0.7258147597312927, 0.9187740087509155, 1.579329252243042, -1.8714854717254639, 0.06533358246088028, 3.2113919258117676, -1.0119222402572632, -1.3378310203552246, -1.1801493167877197, -1.109752893447876, 0.5883309841156006, 0.11524142324924469, 2.2965714931488037, -2.172760486602783, 1.6129295825958252, -1.0404263734817505, 1.166547060012817

In [92]:
print(fragment)

[['frame-108488.mp4', 7, 31, 35.2620739870481], ['frame-068430.mp4', 1, 31, 35.75379182342183], ['frame-079279.mp4', 3, 31, 36.43787188954928], ['frame-061214.mp4', 20, 31, 38.91446268116095], ['frame-079506.mp4', 0, 31, 39.13713424014411], ['frame-011017.mp4', 0, 31, 39.42465830474155], ['frame-092035.mp4', 4, 31, 39.93369058851048], ['frame-061214.mp4', 56, 31, 40.274902006186444]]


In [96]:
# show the original film fragment

#film = VideoFileClip(os.path.join('../processed_films', keyName))
try:
    path = getPoC(keyName, '/processed_films/frame-112191.mp4')
    film = VideoFileClip(path)
except:
    print('fail')
    film = VideoFileClip(os.path.join('processed_films', keyName))

filmFragment = film.subclip(s,s+d)
filmFragment.ipython_display()


Output hidden; open in https://colab.research.google.com to view.

In [99]:
filmFragment.write_videofile("output_films/fragment.mp4", audio_codec='aac')

t:   1%|          | 2/192 [00:42<00:15, 12.17it/s, now=None]

Moviepy - Building video output_films/fragment.mp4.
MoviePy - Writing audio in fragmentTEMP_MPY_wvf_snd.mp4



t:   1%|          | 2/192 [00:42<00:15, 12.17it/s, now=None]

MoviePy - Done.
Moviepy - Writing video output_films/fragment.mp4




t:   1%|          | 2/192 [00:57<00:15, 12.17it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_films/fragment.mp4


In [100]:
fragment

[['frame-108488.mp4', 7, 31, 35.2620739870481],
 ['frame-068430.mp4', 1, 31, 35.75379182342183],
 ['frame-079279.mp4', 3, 31, 36.43787188954928],
 ['frame-061214.mp4', 20, 31, 38.91446268116095],
 ['frame-079506.mp4', 0, 31, 39.13713424014411],
 ['frame-011017.mp4', 0, 31, 39.42465830474155],
 ['frame-092035.mp4', 4, 31, 39.93369058851048],
 ['frame-061214.mp4', 56, 31, 40.274902006186444]]

In [103]:
# extract the fragment from the film

fragmentFilms = []
for f in fragment:
    fragmentFilms.append(extractFragment(f, 'processed_films'))

In [114]:
# display the film fragments

# fragmentFilms[0].ipython_display()

final_clip = clips_array([fragmentFilms[0:3], fragmentFilms[3:6], fragmentFilms[6:9]])

IndexError: ignored

In [115]:
def print_nested_list(lst, indent=0):
    for i, item in enumerate(lst):
        if isinstance(item, list):
            print("  " * indent + f"List {i + 1}:")
            print_nested_list(item, indent + 1)
        else:
            print("  " * indent + f"Item {i + 1}: {item}")

# 打印 fragmentFilms 的结构
print_nested_list(fragmentFilms)



Item 1: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb5072334f0>
Item 2: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca0760>
Item 3: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca03a0>
Item 4: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca1300>
Item 5: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca3a90>
Item 6: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca2e00>
Item 7: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca3c10>
Item 8: <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x7eb4c9ca0fa0>


In [118]:
print(len(fragmentFilms))


8


In [125]:
# 创建 CompositeVideoClip
final_clip = CompositeVideoClip(video_clips)

# 将 CompositeVideoClip 写入文件，禁用音频写入
final_clip.write_videofile("__temp__.mp4", codec='libx264', audio=None)


chunk:  40%|████      | 365/912 [04:47<00:05, 101.20it/s, now=None]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4




t:  36%|███▌      | 353/992 [02:03<03:56,  2.70it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file processed_films/frame-108488.mp4, 2108160 bytes wanted but 0 bytes read,at frame 637/639, at time 26.57/26.65 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+


t:  36%|███▌      | 354/992 [02:04<03:57,  2.68it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file processed_films/frame-108488.mp4, 2108160 bytes wanted but 0 bytes read,at frame 638/639, at time 26.61/26.65 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+


t:  37%|███▋      | 370/992 [02:09<03:33,  2.91it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file proces

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [129]:
from moviepy.editor import CompositeVideoClip

# 假设fragmentFilms是你的VideoFileClip对象列表
video_clips = [
    CompositeVideoClip(fragmentFilms[0:3]).subclip(0, 15),
    CompositeVideoClip(fragmentFilms[3:6]).subclip(0, 15),
    CompositeVideoClip(fragmentFilms[6:9]).subclip(0, 15)
]

final_clip = CompositeVideoClip(video_clips)
final_clip.ipython_display()

Output hidden; open in https://colab.research.google.com to view.

In [140]:
from moviepy.editor import VideoClip

# 拆分 CompositeVideoClip 为 VideoClip 对象
video_clips = [
    fragment.set_duration(10)  # 设置每个片段的时长为15秒
    for fragment in fragmentFilms[0:3]
]

video_clips += [
    fragment.set_duration(10)
    for fragment in fragmentFilms[3:6]
]

video_clips += [
    fragment.set_duration(10)
    for fragment in fragmentFilms[6:9]
]

# 创建 CompositeVideoClip
final_clip = clips_array([video_clips])
final_clip.ipython_display()







chunk:  40%|████      | 365/912 [29:04<00:05, 101.20it/s, now=None]

chunk:  95%|█████████▌| 316/331 [00:33<00:00, 81.17it/s, now=None]



chunk:  40%|████      | 365/912 [29:04<00:05, 101.20it/s, now=None]

chunk:  95%|█████████▌| 316/331 [00:33<00:00, 81.17it/s, now=None]

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3





chunk:   0%|          | 0/221 [00:00<?, ?it/s, now=None]


chunk:   1%|          | 2/221 [00:00<01:11,  3.05it/s, now=None]


chunk:  10%|▉         | 22/221 [00:00<00:05, 38.24it/s, now=None]


chunk:  15%|█▌        | 34/221 [00:00<00:03, 53.18it/s, now=None]


chunk:  22%|██▏       | 48/221 [00:00<00:02, 72.23it/s, now=None]


chunk:  28%|██▊       | 61/221 [00:01<00:02, 62.47it/s, now=None]


chunk:  38%|███▊      | 83/221 [00:01<00:01, 93.47it/s, now=None]


chunk:  45%|████▌     | 100/221 [00:01<00:01, 109.03it/s, now=None]


chunk:  52%|█████▏    | 115/221 [00:01<00:01, 89.82it/s, now=None] 


chunk:  60%|██████    | 133/221 [00:01<00:00, 107.83it/s, now=None]


chunk:  67%|██████▋   | 148/221 [00:01<00:00, 117.15it/s, now=None]


chunk:  74%|███████▍  | 163/221 [00:02<00:00, 89.37it/s, now=None] 


chunk:  84%|████████▎ | 185/221 [00:02<00:00, 114.72it/s, now=None]


chunk:  93%|█████████▎| 205/221 [00:02<00:00, 98.71it/s, now=None] 


chunk: 100%|█████████▉| 220/221 [00:02<00

MoviePy - Done.
Moviepy - Writing video __temp__.mp4






t:   0%|          | 0/240 [00:00<?, ?it/s, now=None]


t:   1%|          | 2/240 [00:04<09:05,  2.29s/it, now=None]


t:   1%|▏         | 3/240 [00:05<06:11,  1.57s/it, now=None]


t:   2%|▏         | 4/240 [00:05<04:37,  1.18s/it, now=None]


t:   2%|▏         | 5/240 [00:06<03:43,  1.05it/s, now=None]


t:   2%|▎         | 6/240 [00:06<03:17,  1.18it/s, now=None]


t:   3%|▎         | 7/240 [00:07<02:59,  1.30it/s, now=None]


t:   3%|▎         | 8/240 [00:07<02:43,  1.42it/s, now=None]


t:   4%|▍         | 9/240 [00:08<02:33,  1.51it/s, now=None]


t:   4%|▍         | 10/240 [00:09<02:27,  1.56it/s, now=None]


t:   5%|▍         | 11/240 [00:09<02:25,  1.58it/s, now=None]


t:   5%|▌         | 12/240 [00:10<02:19,  1.63it/s, now=None]


t:   5%|▌         | 13/240 [00:11<02:21,  1.60it/s, now=None]


t:   6%|▌         | 14/240 [00:11<02:13,  1.70it/s, now=None]


t:   6%|▋         | 15/240 [00:12<02:14,  1.67it/s, now=None]


t:   7%|▋         | 16/240 [00:12<02:12,  1.69it/s, no

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [141]:
final_clip.resize(width=720).ipython_display()





chunk:  40%|████      | 365/912 [32:24<00:05, 101.20it/s, now=None]

chunk:  95%|█████████▌| 316/331 [03:52<00:00, 81.17it/s, now=None]



chunk:  40%|████      | 365/912 [32:24<00:05, 101.20it/s, now=None]

chunk:  95%|█████████▌| 316/331 [03:52<00:00, 81.17it/s, now=None]

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3





chunk:   0%|          | 0/221 [00:00<?, ?it/s, now=None]


chunk:   1%|          | 2/221 [00:00<01:29,  2.43it/s, now=None]


chunk:   7%|▋         | 15/221 [00:00<00:09, 21.25it/s, now=None]


chunk:  13%|█▎        | 29/221 [00:01<00:04, 40.95it/s, now=None]


chunk:  18%|█▊        | 39/221 [00:01<00:03, 48.26it/s, now=None]


chunk:  23%|██▎       | 51/221 [00:01<00:02, 62.45it/s, now=None]


chunk:  28%|██▊       | 61/221 [00:01<00:03, 49.83it/s, now=None]


chunk:  34%|███▍      | 75/221 [00:01<00:02, 66.29it/s, now=None]


chunk:  41%|████      | 90/221 [00:01<00:01, 83.16it/s, now=None]


chunk:  47%|████▋     | 103/221 [00:02<00:01, 62.91it/s, now=None]


chunk:  53%|█████▎    | 117/221 [00:02<00:01, 75.86it/s, now=None]


chunk:  59%|█████▉    | 131/221 [00:02<00:01, 88.65it/s, now=None]


chunk:  65%|██████▍   | 143/221 [00:02<00:00, 95.26it/s, now=None]


chunk:  70%|███████   | 155/221 [00:02<00:01, 65.03it/s, now=None]


chunk:  78%|███████▊  | 172/221 [00:02<00:00, 83.1

MoviePy - Done.
Moviepy - Writing video __temp__.mp4






t:   0%|          | 0/240 [00:00<?, ?it/s, now=None]


t:   1%|          | 2/240 [00:00<01:01,  3.86it/s, now=None]


t:   1%|▏         | 3/240 [00:01<01:29,  2.65it/s, now=None]


t:   2%|▏         | 4/240 [00:01<01:48,  2.18it/s, now=None]


t:   2%|▏         | 5/240 [00:02<01:58,  1.98it/s, now=None]


t:   2%|▎         | 6/240 [00:02<02:01,  1.92it/s, now=None]


t:   3%|▎         | 7/240 [00:03<02:07,  1.83it/s, now=None]


t:   3%|▎         | 8/240 [00:03<02:07,  1.82it/s, now=None]


t:   4%|▍         | 9/240 [00:04<02:07,  1.81it/s, now=None]


t:   4%|▍         | 10/240 [00:05<02:10,  1.77it/s, now=None]


t:   5%|▍         | 11/240 [00:05<02:12,  1.73it/s, now=None]


t:   5%|▌         | 12/240 [00:06<02:12,  1.72it/s, now=None]


t:   5%|▌         | 13/240 [00:06<02:11,  1.72it/s, now=None]


t:   6%|▌         | 14/240 [00:07<02:10,  1.73it/s, now=None]


t:   6%|▋         | 15/240 [00:08<02:11,  1.71it/s, now=None]


t:   7%|▋         | 16/240 [00:08<02:10,  1.71it/s, no

Moviepy - Done !
Moviepy - video ready __temp__.mp4
